# Câu 1:

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from itertools import combinations
from math import sqrt
# Tạo kết nối SQLite
conn = sqlite3.connect(":memory:")
cur = conn.cursor()
# Tạo bảng dữ liệu A, B
cur.execute('''
CREATE TABLE scores (
    id INTEGER PRIMARY KEY,
    A REAL,
    B REAL
)
''')
data = [
    (1, 8, 9),
    (2, 7.5, 8.5),
    (3, 6, 7),
    (4, 7, 6)
]
cur.executemany('INSERT INTO scores (id, A, B) VALUES (?, ?, ?)', data)

query = '''
SELECT
    (COUNT(*) * SUM(A * B) - SUM(A) * SUM(B)) /
    (SQRT(COUNT(*) * SUM(A*A) - SUM(A)*SUM(A)) * SQRT(COUNT(*) * SUM(B*B) - SUM(B)*SUM(B))) AS r_ab
FROM scores
'''
r_ab = cur.execute(query).fetchone()[0]
print("Hệ tương quan Pearson r_AB =", round(r_ab, 4))

Hệ tương quan Pearson r_AB = 0.6911


# Câu 2:

In [2]:
# Tạo bảng điểm thử nghiệm
cur.execute('''
CREATE TABLE test_scores (
    Day TEXT,
    A REAL,
    B REAL,
    C REAL
)
''')

# Chèn dữ liệu
test_data = [
    ('Day 1', 8, 9, 7),
    ('Day 2', 7.5, 8.5, 7),
    ('Day 3', 6, 7, 8),
    ('Day 4', 7, 6, 5)
]
cur.executemany('INSERT INTO test_scores VALUES (?, ?, ?, ?)', test_data)

# Chuyển về dạng quan hệ
query_long_format = '''
SELECT Day, 'A' AS Model, A AS Score FROM test_scores
UNION ALL
SELECT Day, 'B', B FROM test_scores
UNION ALL
SELECT Day, 'C', C FROM test_scores
'''
df_long = pd.read_sql_query(query_long_format, conn)
print("Dữ liệu dạng quan hệ:")
print(df_long)

Dữ liệu dạng quan hệ:
      Day Model  Score
0   Day 1     A    8.0
1   Day 2     A    7.5
2   Day 3     A    6.0
3   Day 4     A    7.0
4   Day 1     B    9.0
5   Day 2     B    8.5
6   Day 3     B    7.0
7   Day 4     B    6.0
8   Day 1     C    7.0
9   Day 2     C    7.0
10  Day 3     C    8.0
11  Day 4     C    5.0


In [3]:
from scipy.stats import chi2_contingency

contingency = pd.crosstab(df_long['Day'], df_long['Model'])

# Kiểm định
chi2, p, dof, expected = chi2_contingency(contingency)

print("\nKết quả kiểm định Chi-squared:")
print("Chi2 =", round(chi2, 4))
print("p-value =", round(p, 4))
print("Degrees of Freedom:", dof)
print("Sự khác biệt đáng kể: " , "Có" if p < 0.05 else "Không có")


Kết quả kiểm định Chi-squared:
Chi2 = 0.0
p-value = 1.0
Degrees of Freedom: 6
Sự khác biệt đáng kể:  Không có


# Câu 3:

In [4]:
cur.execute("create table flights(flight_id integer, departure_time integer)")
flights_data = [(1, 830), (2, 1445), (3, 120), (4, 2359)]
cur.executemany("insert into flights values (?, ?)", flights_data)

# Tạo cột formatted_time từ departure_time (hhmm)
cur.execute("""
    select 
        flight_id,
        departure_time,
        printf('%02d:%02d', departure_time / 100, departure_time % 100) as formatted_time
    from flights
""")
flights_result = cur.fetchall()
df_flights = pd.DataFrame(flights_result, columns=["flight_id", "departure_time", "formatted_time"])
df_flights

,flight_id,departure_time,formatted_time
0,1,830,08:30
1,2,1445,14:45
2,3,120,01:20
3,4,2359,23:59


# Câu 4:

In [6]:
cur.execute('''CREATE TABLE data_2(
           gia_tri float
           )''')
values = [(14,), (5,), (7,), (8,), (-2,), (111,), (10,), (11,), (13,), (5,), (199,)]
cur.executemany("INSERT INTO data_2(gia_tri) VALUES (?)", values)
conn.commit()
cur.execute("""
SELECT gia_tri FROM data_2
ORDER BY gia_tri
""")
vals = [row[0] for row in cur.fetchall()]
n = len(vals)
median = (vals[n // 2] if n % 2 == 1 else (vals[n // 2 - 1] + vals[n // 2]) / 2)

# Tính |xi - median|
abs_devs = [abs(v - median) for v in vals]
abs_devs.sort()
mad = (abs_devs[n // 2] if n % 2 == 1 else (abs_devs[n // 2 - 1] + abs_devs[n // 2]) / 2)

# ngoại lệ (vượt 1.5 × MAD)
threshold = 1.5 * mad
outliers = [v for v in vals if abs(v - median) > threshold]

print(f"Median = {median}")
print(f"MAD = {mad}")
print(f"Ngưỡng ngoại lệ (1.5×MAD): ±{threshold}")
print("Giá trị ngoại lệ:", outliers)

Median = 10.0
MAD = 4.0
Ngưỡng ngoại lệ (1.5×MAD): ±6.0
Giá trị ngoại lệ: [-2.0, 111.0, 199.0]


# Câu 5:

In [7]:

cur.execute("CREATE TABLE Patient(id INTEGER, last_name TEXT, weight REAL, height REAL)")
patient_data = [
    (1, "Nguyen", 60, 170),
    (2, "Nguyen", 62, 172),
    (3, "Tran", 75, 180),
    (4, "Le", 80, 178)
]
cur.executemany("INSERT INTO Patient VALUES (?, ?, ?, ?)", patient_data)

df_patients = pd.read_sql_query("SELECT * FROM Patient", conn)

# Khoảng sai số chấp nhận cho weight (ví dụ ±1kg)
WEIGHT_TOLERANCE = 3.0

pairs = []
for (i, p1), (j, p2) in combinations(df_patients.iterrows(), 2):
    # So sánh theo khoảng cách Boolean:
    same_last_name = p1['last_name'] == p2['last_name']
    similar_weight = abs(p1['weight'] - p2['weight']) <= WEIGHT_TOLERANCE

    if same_last_name and similar_weight:
        pairs.append((p1['id'], p2['id'], p1['last_name'], p1['weight'], p2['weight']))

df_pairs = pd.DataFrame(pairs, columns=["id_1", "id_2", "last_name", "weight_1", "weight_2"])

print(df_patients)

df_pairs

   id last_name  weight  height
0   1    Nguyen    60.0   170.0
1   2    Nguyen    62.0   172.0
2   3      Tran    75.0   180.0
3   4        Le    80.0   178.0


,id_1,id_2,last_name,weight_1,weight_2
0,1,2,Nguyen,60.0,62.0
